# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD


In [2]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_subsampled.txt',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [3]:
config = pd.read_csv('config.csv').iloc[0]

In [4]:
config

Unnamed: 0                             0
id                                276633
k                                      5
dataset_file    ./Behance_subsampled.txt
output_file                 ./output.txt
Name: 0, dtype: object

In [5]:
user = config['id']
k_value = config['k']

### Read the Data

In [6]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [7]:
user_likes = dict()


In [8]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    user_likes[user_id].append(item_id)

In [9]:
user_list = list(user_likes.keys())
item_list = list(user_likes.values())

In [10]:
import itertools

item_list = list(itertools.chain.from_iterable(item_list))

In [11]:
item_list = list(set(item_list))

In [12]:
utility_mat = np.zeros((len(user_list), len(item_list)))

In [13]:
item_index_dict = {}

In [14]:
for i in range(len(item_list)):
    item_index_dict[item_list[i]] = i

In [15]:
user_index_dict = {}

In [16]:
for i in range(len(user_list)):
    user_index_dict[user_list[i]] = i

In [17]:
for i in range(len(user_list)):
    for item in user_likes[user_list[i]]:
        utility_mat[i][item_index_dict[item]] += 1

In [18]:
utility_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
SVD = TruncatedSVD(n_components=500)

In [20]:
svd_mat = SVD.fit_transform(utility_mat)


### Use KNN after Collaborative Filtering to find nearest neighbors

In [21]:
user = user_list[0]
user_latent_vec = svd_mat[user_index_dict[user]]

In [22]:
distance = {}


In [23]:
def get_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

In [24]:
for a_user in user_list:
    distance[a_user] = get_distance(user_latent_vec, svd_mat[user_index_dict[a_user]])
            

In [25]:
def neighbors(user,k_value):
    k_n_n = []
    for i in range(k_value):
        min_distance = 9999999
        min_distance_user = 0
        for a_user in user_list:
            if a_user in k_n_n or a_user == user:
                continue
            elif min_distance >= distance[a_user]:
                min_distance_user = a_user
                min_distance = distance[a_user]
        k_n_n.append(min_distance_user)
    return k_n_n

### Open the output file to write all the lines to the file

In [26]:
knn = neighbors(user, 5)

In [28]:
knn

[2702897, 3039714, 3901739, 3171252, 2149628]

In [29]:
user_list

[10356,
 15341,
 18197,
 19830,
 28270,
 32901,
 33700,
 34491,
 36623,
 37226,
 42343,
 50931,
 57883,
 66735,
 70248,
 75130,
 80052,
 83239,
 84523,
 87325,
 87936,
 90343,
 94964,
 104300,
 104487,
 112326,
 112733,
 117651,
 118510,
 121578,
 126378,
 129642,
 130298,
 137671,
 142741,
 143767,
 143807,
 144448,
 144458,
 148542,
 148915,
 151649,
 154110,
 158099,
 164694,
 170825,
 174918,
 187704,
 187808,
 192014,
 198561,
 205943,
 206088,
 206299,
 209263,
 209778,
 215838,
 216920,
 217785,
 235551,
 243306,
 252568,
 255714,
 256871,
 257507,
 271547,
 278768,
 281039,
 281979,
 285597,
 294598,
 296988,
 298528,
 300928,
 304961,
 309001,
 317895,
 319160,
 322103,
 323463,
 331814,
 333388,
 335605,
 337372,
 351990,
 357477,
 366769,
 366885,
 373749,
 375075,
 387982,
 389504,
 398414,
 399642,
 407864,
 410484,
 412007,
 425002,
 425561,
 428008,
 428668,
 429346,
 435286,
 438061,
 445117,
 445963,
 446987,
 448115,
 451633,
 453316,
 456913,
 459183,
 464530,
 47078

In [27]:
outFile = open(config['output_file'], 'w')
knn = neighbors(user,k_value)
for user_id in knn:
    for item_id in user_likes[user_id]:
        if item_id in user_likes[user]:
            continue
        else:
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()